### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


In [180]:
user_by_movie.iloc[0][68646]

10.0

`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [9]:
a = np.random.rand(10, 4)

In [79]:
a

array([[0.26616716, 0.11356721, 0.91938456, 0.58336671],
       [0.57856958, 0.45538962, 0.91023468, 0.36768318],
       [0.51035063, 0.61621528, 0.81037042, 0.40765957],
       [0.68820504, 0.43984707, 0.99396716, 0.89454521],
       [0.97588826, 0.94626314, 0.31868604, 0.08152939],
       [0.83176103, 0.31862704, 0.5435034 , 0.37280761],
       [0.08652223, 0.36890462, 0.78002631, 0.99114613],
       [0.18733189, 0.96221188, 0.07875088, 0.51141441],
       [0.43177174, 0.47400855, 0.56356438, 0.70669675],
       [0.36637655, 0.30522138, 0.43401726, 0.29350393]])

In [81]:
a[1]

array([0.57856958, 0.45538962, 0.91023468, 0.36768318])

In [250]:
ratings_mat.shape[1]

4

In [251]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = n_users * n_movies # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for user in range(n_users):
            for movie in range(n_movies):
                # print('This is iteration {}, for user {} and movie {}.'.format(iteration, user, movie))
                
                # if the rating exists
                rating = ratings_mat[user][0, movie]
                if not pd.isna(rating):
                    # print('user {}, movie {}, rating {}'.format(user, movie, rating))
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = rating - np.dot(user_mat[user], movie_mat[:, movie])

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum =+ error
                    
                    # update the values in each matrix in the direction of the gradient
                    update_user_mat = user_mat[user] + learning_rate * 2 * error * movie_mat[:, movie]
                    user_mat[user] = update_user_mat
                    update_movie_mat = movie_mat[:, movie] + learning_rate * 2 * error * user_mat[user]
                    movie_mat[:, movie] = update_movie_mat

        # print results for iteration
        try: 
            print('Step: {}, error is: {}.'.format(iteration, error))
        except:
            print('Rating was nan.')
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [253]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimization Statistics
Iterations | Mean Squared Error 
Step: 0, error is: 3.6801889599411877.
Step: 1, error is: 0.6286877143265324.
Step: 2, error is: -0.8290635004693474.
Step: 3, error is: -1.2876426124411697.
Step: 4, error is: -1.431093879875636.
Step: 5, error is: -1.4813522664158025.
Step: 6, error is: -1.5016446386625617.
Step: 7, error is: -1.5117155400825943.
Step: 8, error is: -1.5183196064107989.
Step: 9, error is: -1.52378076281909.


In [254]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.96002609  9.12917843 10.28815062 10.25571629]
 [ 8.8104628   7.29091319  9.35098292  8.53812803]
 [ 7.90154811  6.93593515  8.19775234  7.77163107]
 [ 9.47562019  8.34921519  9.94311618  9.65831199]
 [ 8.62496498  7.10789019  9.2156629   8.45552149]
 [ 6.90599773  5.4077501   7.57753531  6.83381993]
 [ 9.30863653  7.83678748  9.83995933  9.11408425]
 [ 8.07655411  7.06018086  8.57325163  8.36602583]
 [ 8.62160755  7.41746314  8.97609135  8.35138369]
 [ 7.83644444  6.43853491  8.31097877  7.51708642]
 [ 8.92628078  7.83706198  9.27510966  8.84058422]
 [ 9.5658352   8.45232445  9.99011915  9.62767892]
 [ 9.27527265  8.42082138  9.52399572  9.27426161]
 [ 6.5887262   6.054537    6.75080447  6.69239528]
 [ 9.80907481  8.27498537 10.38815497  9.7017513 ]
 [ 9.74478243  8.30321516 10.34312988  9.77644375]
 [ 8.27823649  7.07681886  8.77584922  8.35204953]
 [ 7.08260235  6.86338744  7.02560987  7.12285488]
 [ 8.24797214  7.28387063  8.70821198  8.54078424]
 [ 8.07953443  7.508608    8.21

**Write your summary here.**
<br>
Numbers are not close for most - we started with random numbers, had relative small learning rate and did only 10 iterations. We need to perform more iterations my friend :)

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [132]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
Step: 0, error is: 4.2669616996794435.
Step: 1, error is: 2.1127340473581544.
Step: 2, error is: 0.5765101055042425.
Step: 3, error is: -0.16348408836428074.
Step: 4, error is: -0.45576549379787146.
Step: 5, error is: -0.5602454893542914.
Step: 6, error is: -0.5924868408123531.
Step: 7, error is: -0.5973303147278228.
Step: 8, error is: -0.591872149692966.
Step: 9, error is: -0.5827677030022.
Step: 10, error is: -0.5726831448326895.
Step: 11, error is: -0.5627138481229519.
Step: 12, error is: -0.5533210750594142.
Step: 13, error is: -0.5447086319531866.
Step: 14, error is: -0.5369786635401228.
Step: 15, error is: -0.5301965044412054.
Step: 16, error is: -0.5244167185648774.
Step: 17, error is: -0.5196921540070312.
Step: 18, error is: -0.5160754519782067.
Step: 19, error is: -0.5136171783100494.
Step: 20, error is: -0.5123624547602033.
Step: 21, error is: -0.5123469768301714.
Step: 22, error is: -0.5135928972909332.
Step: 23, error

Step: 213, error is: 0.007595964912782804.
Step: 214, error is: 0.007370883163028807.
Step: 215, error is: 0.007152414530144924.
Step: 216, error is: 0.006940371339867468.
Step: 217, error is: 0.006734570680066554.
Step: 218, error is: 0.006534834324840588.
Step: 219, error is: 0.006340988655948898.
Step: 220, error is: 0.006152864582006146.
Step: 221, error is: 0.005970297455847096.
Step: 222, error is: 0.005793126990427666.
Step: 223, error is: 0.005621197173574899.
Step: 224, error is: 0.005454356181889608.
Step: 225, error is: 0.005292456294101022.
Step: 226, error is: 0.005135353804060827.
Step: 227, error is: 0.0049829089336626.
Step: 228, error is: 0.0048349857458411805.
Step: 229, error is: 0.004691452057845602.
Step: 230, error is: 0.0045521793549658796.
Step: 231, error is: 0.004417042704824681.
Step: 232, error is: 0.004285920672378651.
Step: 233, error is: 0.004158695235768839.
Step: 234, error is: 0.004035251703063736.
Step: 235, error is: 0.0039154786300477085.
Step: 236,

In [133]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.00003189  9.99995204 10.00007437  9.99992937]
 [10.00043346  4.00007338  9.00001279  9.99962567]
 [ 7.99998056  8.99997848 10.00001401  5.00000568]
 [ 9.00195479  8.00019735 10.00023883  9.99819967]
 [ 9.99885097  4.99987472  8.99981783  9.00104991]
 [ 5.99944695  3.99978595 10.00013179  6.00037925]
 [ 8.99735075  7.99945637 10.00003214  9.00220917]
 [ 9.9990808   4.99983733  9.00002182  8.00078839]
 [ 7.00041437  8.00031448  9.99972575  7.99984088]
 [ 9.00243124  5.00046996  9.00003785  6.99794911]]
[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


**Write your summary here.**
<br>
Simply beautiful, so close!

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [227]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [228]:
ratings_mat.max(0)

matrix([[nan, 10., 10., 10.]])

In [230]:
ratings_mat[0].max()

nan

In [135]:
pd.isna(ratings_mat[0, 0])

True

In [136]:
# run SVD on the matrix with the missing value
# use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
Step: 0, error is: 4.391233665977479.
Step: 1, error is: 2.3121122989311598.
Step: 2, error is: 0.7086055687363331.
Step: 3, error is: -0.11091946588369339.
Step: 4, error is: -0.4568558299241392.
Step: 5, error is: -0.5973306826328564.
Step: 6, error is: -0.6563280479408355.
Step: 7, error is: -0.6829683546259515.
Step: 8, error is: -0.6965076422005758.
Step: 9, error is: -0.7048009857173998.
Step: 10, error is: -0.7112125158417708.
Step: 11, error is: -0.7172290690608873.
Step: 12, error is: -0.7235020084981336.
Step: 13, error is: -0.7302915452996688.
Step: 14, error is: -0.7376686403778674.
Step: 15, error is: -0.7456111102155694.
Step: 16, error is: -0.7540505727054878.
Step: 17, error is: -0.7628956277685432.
Step: 18, error is: -0.772043431776166.
Step: 19, error is: -0.7813857306752006.
Step: 20, error is: -0.7908124054586123.
Step: 21, error is: -0.8002140202834429.
Step: 22, error is: -0.8094840319092267.
Step: 23, erro

In [137]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.648619813167262:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [255]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

In [256]:
# perform funkSVD on the matrix of the top 1000 users
# fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimization Statistics
Iterations | Mean Squared Error 
Step: 0, error is: 7.247610347746272.


KeyboardInterrupt: 

`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [244]:
np.count_nonzero(~np.isnan(first_1000_users))

10852

In [245]:
np.count_nonzero(np.isnan(first_1000_users))

31234148

In [246]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = np.count_nonzero(np.isnan(first_1000_users)) # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [247]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?


In [248]:
preds[0]

array([3.74033012, 4.94486014, 0.66627784, 3.16592169, 4.49112556,
       5.00876222, 3.39696872, 3.6657169 , 5.40947428, 4.1159386 ,
       4.4018173 , 4.58885307, 3.86188214, 3.25094603, 4.6970192 ,
       3.24655031, 2.64306005, 3.66855813, 4.57022451, 2.61210266,
       2.86576515, 4.38112949, 3.38543484, 1.74412368, 4.22523925,
       3.58989305, 3.29119132, 4.09302189, 1.23221831, 3.676169  ,
       4.19279347, 1.2962601 , 4.09357197, 3.86940371, 3.13508826,
       3.81647665, 2.03955725, 4.54106519, 5.34148027, 2.90238875,
       3.51398464, 3.0652785 , 3.97709367, 4.10023563, 4.11460351,
       4.93599885, 1.90907908, 2.92733953, 3.60504865, 3.38251359,
       3.7992467 , 3.0666771 , 5.45996996, 1.94472252, 2.98261432,
       5.71270958, 3.36357314, 4.7805007 , 3.84862121, 3.36379608,
       2.25840502, 5.1016377 , 4.58569253, 3.18744304, 2.92391593,
       1.32703063, 2.21895214, 2.12795858, 3.93008989, 3.22763019,
       2.39442811, 2.03499007, 3.74161154, 4.4285808 , 4.62558